# Import nescessary libraries

In [ ]:
# Import necessary libraries

import random
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Check 'CPU' or 'GPU'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
# Hyperparameters

n_classes = 10
learning_rate = 0.001
batch_size = 64
n_epochs = 5

In [ ]:
transforms = transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor()])

# Prepare MNIST dataset

In [ ]:
#Load data to our Google Drive

train_dataset = datasets.MNIST(root='/content/drive/MyDrive/10. Toán cho KHMT/As 5/dataset_mnist', train=True,transform=transforms,download= True)

train_loader = DataLoader(dataset = train_dataset, batch_size=batch_size, shuffle = True)
test_dataset = datasets.MNIST(root='/content/drive/MyDrive/10. Toán cho KHMT/As 5/dataset_mnist',train=False,transform=transforms,download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Build LeNet5 model

In [ ]:
# Build LeNet5 by using Pytorch
class LeNet5(nn.Module):
  def __init__(self, n_classes):
    super().__init__()
    self.model = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride =1),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=2),
        nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=2),
        nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
        nn.Tanh(),
        nn.Flatten(),
        nn.Linear(in_features=120, out_features=84),
        nn.Tanh(),
        nn.Linear(in_features= 84, out_features=n_classes),
        nn.Softmax(dim=1)
    )
  def forward(self, X):
    prob = self.model(X)
    return prob

In [ ]:
model = LeNet5(n_classes=n_classes).to(device)
print(model)

LeNet5(
  (model): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (7): Tanh()
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=120, out_features=84, bias=True)
    (10): Tanh()
    (11): Linear(in_features=84, out_features=10, bias=True)
    (12): Softmax(dim=1)
  )
)


In [ ]:
# Define the loss and the optimization algorithm

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= learning_rate)

# Training model

In [ ]:
#Training model

for epoch in range(n_epochs):
  for batch_idx, (data, targets) in enumerate(train_loader):

    data = data.to(device)
    targets = targets.to(device)

    scores = model(data)
    loss = criterion(scores, targets)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    if(batch_idx+1)% 100 ==0:
      print(f'Epoch {epoch+1}/{n_epochs}, Batch {batch_idx+1}, Loss: {loss.item():.2f}')

Epoch 1/5, Batch 100, Loss: 1.71
Epoch 1/5, Batch 200, Loss: 1.57
Epoch 1/5, Batch 300, Loss: 1.53
Epoch 1/5, Batch 400, Loss: 1.58
Epoch 1/5, Batch 500, Loss: 1.48
Epoch 1/5, Batch 600, Loss: 1.51
Epoch 1/5, Batch 700, Loss: 1.47
Epoch 1/5, Batch 800, Loss: 1.48
Epoch 1/5, Batch 900, Loss: 1.51
Epoch 2/5, Batch 100, Loss: 1.54
Epoch 2/5, Batch 200, Loss: 1.55
Epoch 2/5, Batch 300, Loss: 1.50
Epoch 2/5, Batch 400, Loss: 1.52
Epoch 2/5, Batch 500, Loss: 1.50
Epoch 2/5, Batch 600, Loss: 1.49
Epoch 2/5, Batch 700, Loss: 1.48
Epoch 2/5, Batch 800, Loss: 1.51
Epoch 2/5, Batch 900, Loss: 1.48
Epoch 3/5, Batch 100, Loss: 1.48
Epoch 3/5, Batch 200, Loss: 1.46
Epoch 3/5, Batch 300, Loss: 1.48
Epoch 3/5, Batch 400, Loss: 1.46
Epoch 3/5, Batch 500, Loss: 1.49
Epoch 3/5, Batch 600, Loss: 1.49
Epoch 3/5, Batch 700, Loss: 1.46
Epoch 3/5, Batch 800, Loss: 1.49
Epoch 3/5, Batch 900, Loss: 1.47
Epoch 4/5, Batch 100, Loss: 1.47
Epoch 4/5, Batch 200, Loss: 1.46
Epoch 4/5, Batch 300, Loss: 1.53
Epoch 4/5,

# Performance Evaluation

In [ ]:
def get_accracy(loader, model):
  if loader.dataset.train:
    print('Getting accuracy on training data.')
  else:
    print('Getting accuracy on testing data.')
  
  n_corrects = 0
  n_samples = 0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x = x.to(device)
      y = y.to(device)

      scores = model(x)
      _,y_pred = scores.max(1)
      n_corrects += (y_pred == y).sum()
      n_samples += y_pred.size(0)

    print(f'We got {n_corrects}/{n_samples} correct. Accuracy = {float(n_corrects)/float(n_samples)* 100.0:.2f}')
  model.train()

In [ ]:
get_accracy(train_loader,model)
get_accracy(test_loader, model)

Getting accuracy on training data.
We got 59084/60000 correct. Accuracy = 98.47
Getting accuracy on testing data.
We got 9796/10000 correct. Accuracy = 97.96
